In [1]:
import pandas as pd

pd.set_option("display.max_rows", 999)

## State-Bird Stats

In [2]:
# https://ebird.org/science/status-and-trends/download-data
# Regional range and abundance as tabular data
stats = pd.read_csv("/Users/camen/Downloads/all-stats-regional-2021.csv/all-stats-regional-2021.csv")

### Distinct on bird-state

In [3]:
stats = (
    stats
        .loc[((stats.version_year == 2021) & (stats.region_type == "state")), :]
        .sort_values(by="abundance_mean", ascending=False)
        .groupby(by=["region_code", "species_code"])
        .first()
        .reset_index()
)

### Some cleaning

In [4]:
list(stats[stats.common_name.str.contains("/")].common_name.unique())

['Eurasian/Cyprus Scops-Owl',
 'Common/Short-billed Gull',
 'Rufous-capped/Chestnut-capped Warbler',
 'Broad-billed/Turquoise-crowned Hummingbird']

In [5]:
list(stats[stats.scientific_name.str.contains("/")].scientific_name.unique())

['Otus scops/cyprius',
 'Larus canus/brachyrhynchus',
 'Basileuterus rufifrons/delattrii',
 'Cynanthus latirostris/doubledayi']

In [6]:
stats["common_name"] = stats["common_name"].replace("Eurasian/Cyprus Scops-Owl", "Cyprus Scops-Owl")
stats["scientific_name"] = stats["scientific_name"].replace("Otus scops/cyprius", "Otus cyprius")

stats["common_name"] = stats["common_name"].replace("Common/Short-billed Gull", "Short-Billed Gull")
stats["scientific_name"] = stats["scientific_name"].replace("Larus canus/brachyrhynchus", "Larus brachyrhynchus")

stats["common_name"] = stats["common_name"].replace("Rufous-capped/Chestnut-capped Warbler", "Chestnut-capped Warbler")
stats["scientific_name"] = stats["scientific_name"].replace("Basileuterus rufifrons/delattrii", "Basileuterus delattrii")

stats["common_name"] = stats["common_name"].replace("Broad-billed/Turquoise-crowned Hummingbird", "Turquoise-crowned Hummingbird")
stats["scientific_name"] = stats["scientific_name"].replace("Cynanthus latirostris/doubledayi", "Cynanthus doubledayi")


def to_programmatic_name(common_name):
    return common_name.str.replace(r"[-| ]{1}", "_", regex=True).str.replace("'", "").str.lower()


stats["programmatic_name"] = to_programmatic_name(stats.common_name)

In [114]:
stats.head()

,region_code,species_code,taxon_order,common_name,scientific_name,version_year,region_type,region_name,region_area_km2,season_name,start_dt,end_dt,abundance_mean,total_pop_percent,range_percent_occupied,range_total_percent,range_days_occupation,programmatic_name,genus,species
319071,USA-AK,aldfly,16284,Alder Flycatcher,Empidonax alnorum,2021,state,Alaska,1.505332e+06,breeding,2020-06-21,2020-07-13,0.2251,0.0849,0.4466,0.1151,28.0,alder_flycatcher,empidonax,alnorum
319072,USA-AK,aleter1,6464,Aleutian Tern,Onychoprion aleuticus,2021,state,Alaska,1.505332e+06,breeding,2020-05-31,2020-07-20,0.0100,0.6941,0.0120,0.5887,56.0,aleutian_tern,onychoprion,aleuticus
319073,USA-AK,amecro,20762,American Crow,Corvus brachyrhynchos,2021,state,Alaska,1.505332e+06,nonbreeding,2020-12-14,2020-01-25,0.1616,0.0147,0.0193,0.0065,49.0,american_crow,corvus,brachyrhynchos
319074,USA-AK,amedip,26751,American Dipper,Cinclus mexicanus,2021,state,Alaska,1.505332e+06,year_round,2020-01-04,2020-12-28,0.0054,0.1074,0.0228,0.0917,365.0,american_dipper,cinclus,mexicanus
319075,USA-AK,amekes,11494,American Kestrel,Falco sparverius,2021,state,Alaska,1.505332e+06,postbreeding_migration,2020-08-17,2020-11-09,0.0077,0.0023,0.0647,0.0051,35.0,american_kestrel,falco,sparverius


### Only US

In [8]:
stats = stats.loc[stats.region_code.str.startswith("USA-")]

In [9]:
stats.head()

,region_code,species_code,taxon_order,common_name,scientific_name,version_year,region_type,region_name,region_area_km2,season_name,start_dt,end_dt,abundance_mean,total_pop_percent,range_percent_occupied,range_total_percent,range_days_occupation,programmatic_name
319071,USA-AK,aldfly,16284,Alder Flycatcher,Empidonax alnorum,2021,state,Alaska,1.505332e+06,breeding,2020-06-21,2020-07-13,0.2251,0.0849,0.4466,0.1151,28.0,alder_flycatcher
319072,USA-AK,aleter1,6464,Aleutian Tern,Onychoprion aleuticus,2021,state,Alaska,1.505332e+06,breeding,2020-05-31,2020-07-20,0.0100,0.6941,0.0120,0.5887,56.0,aleutian_tern
319073,USA-AK,amecro,20762,American Crow,Corvus brachyrhynchos,2021,state,Alaska,1.505332e+06,nonbreeding,2020-12-14,2020-01-25,0.1616,0.0147,0.0193,0.0065,49.0,american_crow
319074,USA-AK,amedip,26751,American Dipper,Cinclus mexicanus,2021,state,Alaska,1.505332e+06,year_round,2020-01-04,2020-12-28,0.0054,0.1074,0.0228,0.0917,365.0,american_dipper
319075,USA-AK,amekes,11494,American Kestrel,Falco sparverius,2021,state,Alaska,1.505332e+06,postbreeding_migration,2020-08-17,2020-11-09,0.0077,0.0023,0.0647,0.0051,35.0,american_kestrel


## Birds

In [10]:
birds = stats.groupby("species_code")[["common_name", "scientific_name", "programmatic_name", "taxon_order"]].first().reset_index()
birds[["genus", "species"]] = birds["scientific_name"].str.split(" ", expand=True)

In [11]:
birds.head()

,species_code,common_name,scientific_name,programmatic_name,taxon_order,genus,species
0,abetow,Abert's Towhee,Melozone aberti,aberts_towhee,32254,Melozone,aberti
1,acafly,Acadian Flycatcher,Empidonax virescens,acadian_flycatcher,16283,Empidonax,virescens
2,acowoo,Acorn Woodpecker,Melanerpes formicivorus,acorn_woodpecker,10655,Melanerpes,formicivorus
3,aldfly,Alder Flycatcher,Empidonax alnorum,alder_flycatcher,16284,Empidonax,alnorum
4,aleter1,Aleutian Tern,Onychoprion aleuticus,aleutian_tern,6464,Onychoprion,aleuticus


## Images

### Clean into new directory

In [100]:
import shutil
import os
import warnings
import glob

from PIL import Image, ExifTags
from tqdm import tqdm


def parse_image(filepath, bird=None):
    image = Image.open(filepath)
    width, height = image.size
    
    return dict(
        programmatic_name=bird,
        filepath=filepath,
        width=width,
        height=height,
        max_dimension=max([width, height])
    )


def write_to_filepath(row):
    idx = row.name
    programmatic_name = row.programmatic_name
    width = row.width
    height = row.height
    return f"{programmatic_name}/w-{width}_h-{height}_{idx}.jpg"
    
    


# all_bird_filepaths = {}
# for fp in glob.glob("/Users/camen/Downloads/birdsnap_500/download/images/*/*.jpg"):
#     bird_name = fp.split("/")[-2].lower()
#     bird_filepaths = all_bird_filepaths.get(bird_name, [])
#     bird_filepaths.append(fp)
#     all_bird_filepaths[bird_name] = bird_filepaths
    
# for fp in glob.glob("/Users/camen/Downloads/caltech_ucsd_birds_200_images/*/*.jpg"):
#     bird_name = fp.split("/")[-2].split(".")[-1].lower()
#     bird_filepaths = all_bird_filepaths.get(bird_name, [])
#     bird_filepaths.append(fp)
#     all_bird_filepaths[bird_name] = bird_filepaths
    
# for fp in set(
#     glob.glob("/Users/camen/Downloads/kaggle_birds_400/train/*/*.jpg") +
#     glob.glob("/Users/camen/Downloads/kaggle_birds_400/test/*/*.jpg") + 
#     glob.glob("/Users/camen/Downloads/kaggle_birds_400/valid/*/*.jpg")
# ):
#     bird_name = fp.split("/")[-2].lower().replace(" ", "_").replace("&", "and")
#     bird_filepaths = all_bird_filepaths.get(bird_name, [])
#     bird_filepaths.append(fp)
#     all_bird_filepaths[bird_name] = bird_filepaths
all_bird_filepaths = {
    bird: glob.glob(f"/Users/camen/Downloads/clean_images/{bird}/*.jpg")
    for bird in os.listdir("/Users/camen/Downloads/clean_images/")
}
    

images = pd.concat(
    [
        pd.DataFrame.from_records(
            [
                parse_image(fp, bird=bird) for fp in images
            ]
        )
        for bird, images in all_bird_filepaths.items()
    ]
)

images = images.reset_index()
images = images.loc[images.sort_values(by="max_dimension", ascending=False).groupby("programmatic_name").head(20).index]
images = images.rename(columns={"filepath": "from_filepath"})
images["to_filepath"] = images.apply(write_to_filepath, axis=1)
images["delete"] = False

# # for idx, row in tqdm(images.iterrows(), total=len(images)):
# #     bird = row["bird"]
# #     os.makedirs(os.path.dirname(row["to_filepath"]), exist_ok=True)
# #     shutil.copyfile(row["from_filepath"], os.path.join("/Users/camen/Downloads/clean_images", row["to_filepath"])
                    
images = images[["programmatic_name", "max_dimension", "width", "height", "to_filepath", "delete"]].rename(columns={"to_filepath": "filepath"})

In [102]:
# for _, row in images.iterrows():
#     from_filepath = os.path.join(BASE_DIR, row.from_filepath)
#     to_filepath = os.path.join(BASE_DIR, row.to_filepath)
#     if os.path.exists(from_filepath):
#         os.rename(from_filepath, to_filepath)

In [369]:
images.head()

,programmatic_name,max_dimension,width,height,focal_length,iso,filepath,delete
686,herring_gull,10973,10973,7315,300.0,100.0,herring_gull/dim-10973_iso-100_fl-300_0.jpg,False
9289,limpkin,10146,10146,6764,277.0,400.0,limpkin/dim-10146_iso-400_fl-277_1.jpg,False
2342,american_bittern,8820,8820,5880,300.0,400.0,american_bittern/dim-8820_iso-400_fl-300_2.jpg,False
6908,western_bluebird,7800,7800,7800,78.0,100.0,western_bluebird/dim-7800_iso-100_fl-78_3.jpg,False
5754,yellow_breasted_chat,7532,7532,5120,500.0,800.0,yellow_breasted_chat/dim-7532_iso-800_fl-500_4...,False


## Coverage

In [341]:
stats.groupby("region_code")["programmatic_name"].apply(
    lambda state_birds: (
        1 - round(
            (
                len(set(state_birds) - set(images.programmatic_name.unique())) / 
                len(state_birds)
            ),
            2
        ) 
    ) * 100
)

region_code
USA-AK    89.0
USA-AL    95.0
USA-AR    95.0
USA-AZ    85.0
USA-CA    86.0
USA-CO    92.0
USA-CT    95.0
USA-DC    98.0
USA-DE    96.0
USA-FL    89.0
USA-GA    95.0
USA-HI    86.0
USA-IA    95.0
USA-ID    93.0
USA-IL    95.0
USA-IN    95.0
USA-KS    95.0
USA-KY    96.0
USA-LA    94.0
USA-MA    94.0
USA-MD    96.0
USA-ME    95.0
USA-MI    95.0
USA-MN    96.0
USA-MO    95.0
USA-MS    95.0
USA-MT    94.0
USA-NC    94.0
USA-ND    96.0
USA-NE    96.0
USA-NH    96.0
USA-NJ    95.0
USA-NM    87.0
USA-NV    91.0
USA-NY    93.0
USA-OH    96.0
USA-OK    94.0
USA-OR    93.0
USA-PA    96.0
USA-RI    97.0
USA-SC    97.0
USA-SD    95.0
USA-TN    95.0
USA-TX    87.0
USA-UT    92.0
USA-VA    95.0
USA-VT    96.0
USA-WA    93.0
USA-WI    95.0
USA-WV    98.0
USA-WY    94.0
Name: programmatic_name, dtype: float64

## Database

In [106]:
images

,index,programmatic_name,from_filepath,width,height,max_dimension,to_filepath,delete
0,0,hairy_woodpecker,/Users/camen/Downloads/clean_images/hairy_wood...,794,1200,1200,hairy_woodpecker/w-794_h-1200_0.jpg,False
6719,19,lesser_goldfinch,/Users/camen/Downloads/clean_images/lesser_gol...,1200,900,1200,lesser_goldfinch/w-1200_h-900_6719.jpg,False
6712,12,lesser_goldfinch,/Users/camen/Downloads/clean_images/lesser_gol...,1200,857,1200,lesser_goldfinch/w-1200_h-857_6712.jpg,False
6713,13,lesser_goldfinch,/Users/camen/Downloads/clean_images/lesser_gol...,1200,900,1200,lesser_goldfinch/w-1200_h-900_6713.jpg,False
6714,14,lesser_goldfinch,/Users/camen/Downloads/clean_images/lesser_gol...,1200,960,1200,lesser_goldfinch/w-1200_h-960_6714.jpg,False
...,...,...,...,...,...,...,...,...
9038,18,gray_partridge,/Users/camen/Downloads/clean_images/gray_partr...,224,224,224,gray_partridge/w-224_h-224_9038.jpg,False
9039,19,gray_partridge,/Users/camen/Downloads/clean_images/gray_partr...,224,224,224,gray_partridge/w-224_h-224_9039.jpg,False
1660,0,common_poorwill,/Users/camen/Downloads/clean_images/common_poo...,224,224,224,common_poorwill/w-224_h-224_1660.jpg,False
1661,1,common_poorwill,/Users/camen/Downloads/clean_images/common_poo...,224,224,224,common_poorwill/w-224_h-224_1661.jpg,False


In [117]:
import psycopg2
from sqlalchemy import create_engine

with psycopg2.connect("dbname=bird_herd user=camen") as connection:
    with connection.cursor() as cursor:
        cursor.execute(
            """
            DROP TABLE IF EXISTS birds;
            CREATE TABLE birds (
                common_name text,
                scientific_name text,
                programmatic_name text,
                genus text,
                species text
            );

            DROP TABLE IF EXISTS images;
            CREATE TABLE images (
                programmatic_name text,
                filepath text,
                width integer,
                height integer,
                max_dimension integer,
                delete boolean
            );

            DROP TABLE IF EXISTS stats;
            CREATE TABLE stats (
                programmatic_name text,
                common_name text,
                scientific_name text,
                version_year text,
                region_code text,
                abundance_mean float,
                total_pop_percent float
            )
            """
        )
    connection.commit()

    
engine = create_engine("postgresql://camen@localhost/bird_herd")
birds[
    ["common_name", "scientific_name", "programmatic_name", "genus", "species"]
].to_sql("birds", con=engine, if_exists="append", method="multi", index=False)
images[
    ["programmatic_name", "filepath", "width", "height", "max_dimension", "delete"]
].to_sql("images", con=engine, if_exists="append", method="multi", index=False)
stats[
    ["programmatic_name", "common_name", "scientific_name", "version_year", "region_code", "abundance_mean", "total_pop_percent"]
].to_sql("stats", con=engine, if_exists="append", method="multi", index=False)

17296

In [396]:
# with psycopg2.connect("dbname=bird_herd user=camen", cursor_factory=psycopg2.extras.RealDictCursor) as connection:
#     with connection.cursor() as cursor:
#         query = """
#         UPDATE images
#         SET delete = False
#         WHERE filepath = %s
#         RETURNING programmatic_name
#         ;
#     """
#         cursor.execute(query, ("herring_gull/dim-10973_iso-100_fl-300_0.jpg",))
#         if cursor.rowcount > 0:
#             bird = cursor.fetchone()["programmatic_name"]
#         else:
#             bird = None

In [12]:
# import psycopg2
# from sqlalchemy import create_engine
# import pandas as pd

# engine = create_engine("postgresql://camen@localhost/bird_herd")
# images = pd.read_sql("SELECT * FROM images", engine)

# from PIL import Image
# import os
# import numpy as np
# from tqdm import tqdm

# BASE_DIR = "/Users/camen/Downloads/clean_images/"
# SIZE = 1200
# QUALITY = 70

# df = images.loc[images.max_dimension > SIZE]

# def process(filepath):
#     try:
#         filepath = os.path.join(BASE_DIR, row.filepath)
#         image = Image.open(filepath)
#         resized_image = image.resize((np.array(image.size) / (max((max(image.size) / 1200, 1)))).astype(int))
#         resized_image.save(filepath, quality=QUALITY)
#         return None
#     except Execption as error:
#         return str(error)

# df.filepath.apply(process)    